In [11]:
# Multi-Horizon Portfolio Optimizer with Behavioral Utility
# Author: [N'Adoi Aboagye]
# Inspired by multi-period portfolio optimization for sovereign wealth funds

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from datetime import datetime, timedelta
import streamlit as st

# -----------------------------
# Step 1: Data Construction
# -----------------------------

# 1.1 - Define asset list 
TICKERS = ['SOFI', 'HOOD', 'SBET', 'NU', 'GOOGL', 'TSLA', 'AMZN', 'NVDA']

# 1.2 - Get mixed-frequency historical data
def get_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)
    if isinstance(data.columns, pd.MultiIndex):
        data = data['Close']
    else:
        data = data[['Close']]
    data = data.dropna()
    return data

end_date = datetime.today()
start_date = end_date - timedelta(days=5*365)  # 5 years
price_data = get_data(TICKERS, start_date, end_date)
monthly_returns = price_data.resample('M').last().pct_change().dropna()
daily_returns = price_data.pct_change().dropna()

# 1.3 - Imposed long-term drift (Bayesian adjusted expectations)
drift_adjustments = {
    'SOFI': 0.0017, 'HOOD': 0.0015, 'SBET': 0.0025, 'NU': 0.0017,
    'GOOGL': 0.001, 'TSLA': 0.0015, 'AMZN': 0.0012, 'NVDA': 0.0018
}

adjusted_monthly_returns = monthly_returns.copy()
for t in adjusted_monthly_returns.columns:
    adjusted_monthly_returns[t] += drift_adjustments.get(t, 0)

# 1.4 - Add synthetic shock scenarios (drawdowns)
def simulate_shock_events(returns, severity=0.3, shock_days=5):
    shocks = []
    for i in range(10):
        shock = returns.sample(shock_days) * -severity
        shocks.append(shock)
    return pd.concat(shocks)

shock_returns = simulate_shock_events(daily_returns)

# ------------------------------------
# Step 2: Utility Functions
# ------------------------------------
def loss_averse_utility(returns, threshold=-0.02, loss_aversion=4):
    utility = np.where(returns > threshold, returns, returns * loss_aversion)
    return utility.mean()

def sharpe_utility(returns, rf=0.0):
    excess_returns = returns - rf
    return excess_returns.mean() / excess_returns.std()

def sortino_utility(returns, rf=0.0, target=0):
    downside = returns[returns < target]
    downside_std = np.std(downside) if len(downside) > 0 else 1
    return (returns.mean() - rf) / downside_std

def exponential_utility(returns, risk_aversion=5):
    return -np.mean(np.exp(-risk_aversion * returns))

def log_utility(returns):
    return np.mean(np.log(1 + returns))

# Improved Aboagye Utility
# Prioritizes upside, skewness, and penalizes major drawdowns or failure to exceed a benchmark floor

def aboagye_utility(returns, min_floor=-0.20, alpha=0.7, beta=0.3):
    cumulative = (1 + returns).cumprod()
    drawdown = np.maximum.accumulate(cumulative) - cumulative
    max_drawdown = np.max(drawdown)

    mean_return = np.mean(returns)
    std_dev = np.std(returns)
    skew = pd.Series(returns).skew()

    growth_score = mean_return * (1 + skew)
    penalty = 1 + max(0, abs(min(cumulative[-1] - (1 + min_floor))))

    return alpha * (growth_score / (1 + std_dev)) - beta * penalty

utility_map = {
    "Loss-Averse": loss_averse_utility,
    "Sharpe Ratio": sharpe_utility,
    "Sortino Ratio": sortino_utility,
    "Exponential Utility": exponential_utility,
    "Log Utility": log_utility,
    "Aboagye Ratio": aboagye_ratio
}

# ------------------------------------
# Step 3: Full-Scale Optimization
# ------------------------------------
def full_scale_optimization(return_data, utility_fn, num_portfolios=2000):
    num_assets = return_data.shape[1]
    results = []
    weights_list = []

    for _ in range(num_portfolios):
        weights = np.random.dirichlet(np.ones(num_assets), size=1)[0]
        port_returns = return_data @ weights
        u = utility_fn(port_returns)
        results.append(u)
        weights_list.append(weights)

    best_idx = np.argmax(results)
    best_weights = weights_list[best_idx]
    return best_weights, results[best_idx]

# Run optimization for each horizon

def optimize_for_all_horizons(utility_fn):
    horizons = {
        '1 Month': adjusted_monthly_returns.tail(1),
        '6 Months': adjusted_monthly_returns.tail(6),
        '1 Year': adjusted_monthly_returns.tail(12),
        '3 Years': adjusted_monthly_returns.tail(36),
        '5 Years': adjusted_monthly_returns
    }

    optimal_allocations = {}
    for label, data in horizons.items():
        best_weights, util = full_scale_optimization(data, utility_fn)
        optimal_allocations[label] = dict(zip(data.columns, best_weights.round(4)))
    return optimal_allocations

# --------------------
# Streamlit Interface
# --------------------
st.set_page_config(layout="wide")
st.title("Multi-Horizon Portfolio Optimizer")
st.write("""
This tool uses a full-scale optimization technique with selectable utility functions
to determine optimal portfolio weights over multiple horizons. It accounts for:
- Mixed-frequency data
- Simulated shock events
- Forward-looking return expectations
- Loss aversion, risk-adjusted return, downside risk, and more
""")

selected_utility = st.selectbox("Select Utility Function", list(utility_map.keys()))

if st.button("Run Optimization"):
    utility_fn = utility_map[selected_utility]
    results = optimize_for_all_horizons(utility_fn)
    for horizon, weights in results.items():
        st.subheader(f"Optimal Allocation for {horizon} using {selected_utility}")
        df = pd.DataFrame(list(weights.items()), columns=["Stock", "Weight"])
        st.dataframe(df.set_index("Stock"))

        fig, ax = plt.subplots()
        sns.barplot(x="Weight", y="Stock", data=df.sort_values("Weight", ascending=True), ax=ax)
        st.pyplot(fig)

# --------------------
# Save to CSV for GitHub
# --------------------
def save_results(results):
    df_all = pd.DataFrame()
    for horizon, weights in results.items():
        df = pd.DataFrame.from_dict(weights, orient='index', columns=[horizon])
        df_all = pd.concat([df_all, df], axis=1)
    df_all.to_csv("optimized_weights_by_horizon.csv")

# Example usage: save_results(results) if needed


C:\Users\u35524na\AppData\Local\Temp\ipykernel_8588\1786318129.py:23: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date)
[*********************100%***********************]  8 of 8 completed


SyntaxError: invalid syntax (421834696.py, line 1)